In [1]:
import pandas as pd 
import numpy as np 
import os 
from sklearn.preprocessing import LabelEncoder 
from sklearn.metrics import accuracy_score, precision_score, recall_score, confusion_matrix, f1_score,roc_auc_score,mean_squared_error
import warnings
warnings.filterwarnings(action='ignore')
import random
from lightgbm import LGBMClassifier
from lightgbm import LGBMRegressor

In [2]:
train_data=pd.read_csv('/home/iai/son/lastcheck/spambase_train_data.csv')
test_data=pd.read_csv('/home/iai/son/lastcheck/spambase_test_data.csv')

In [3]:
train_stage_features=['0.7', '278', '61', '0', '0.34', '0.30', '0.5', '0.44', '0.40', '0.778',
                        '0.24', '1.93', '0.39', '1.29', '0.43', '0.2', '0.36', '0.17', '0.20', 
                        '0.28', '0.21', '0.32.1', '0.41', '0.29', '3.756', '0.35', '0.25', '0.15', 
                        '0.64.1', '0.38', '0.45', '0.6', '0.33', '0.42', '0.14', '0.9', '0.10', 
                        '0.3', '0.13', '0.8', '0.19', '0.23', '0.18', '0.22', '0.16', '0.64.2', 
                        '0.1', '0.31', '0.27', '0.11', '0.4', '0.12', '0.26', '0.64', '0.37', '0.32', '0.96']
test_stage_features=['0.7', '278', '61', '0', '0.34', '0.30', '0.5', '0.44', '0.40', '0.778',
                        '0.24', '1.93', '0.39', '1.29', '0.43', '0.2', '0.36', '0.17', '0.20']

In [4]:
train_X=train_data.loc[:,train_stage_features]
train_y=train_data['1']
new_test_data_X=test_data.loc[:,test_stage_features]
new_test_data_y=test_data['1']

In [5]:
ratio=train_y.value_counts()[0]/train_y.value_counts()[1]

In [6]:
# train에는 있는데 test data에는 없는 컬럼 찾기

train_data_columns=np.array(train_X.columns)
test_data_columns=np.array(new_test_data_X.columns)
np.setdiff1d(train_data_columns,test_data_columns)

array(['0.1', '0.10', '0.11', '0.12', '0.13', '0.14', '0.15', '0.16',
       '0.18', '0.19', '0.21', '0.22', '0.23', '0.25', '0.26', '0.27',
       '0.28', '0.29', '0.3', '0.31', '0.32', '0.32.1', '0.33', '0.35',
       '0.37', '0.38', '0.4', '0.41', '0.42', '0.45', '0.6', '0.64',
       '0.64.1', '0.64.2', '0.8', '0.9', '0.96', '3.756'], dtype=object)

In [7]:
import optuna
from lightgbm import LGBMRegressor
from sklearn.model_selection import cross_val_score

test_data_columns=test_stage_features
y='0.28'

train_X_new=train_X.loc[:,test_data_columns]
train_y_new=train_X.loc[:,y]



# Objective 함수 정의
def objective(trial):
    
    # 하이퍼파라미터 탐색할 공간 정의
    params = {
        'n_estimators': trial.suggest_int('n_estimators',100,1000),
        'max_depth': trial.suggest_int('max_depth',3,9),
        'learning_rate': trial.suggest_loguniform('learning_rate',0.01,0.1),
    }
    
    # LGBMRegressor 모델 객체 생성
    model = LGBMRegressor(**params, random_state=42)
    
    # 교차검증 수행하여 모델 성능 측정
    scores = -1 * cross_val_score(model, train_X_new, train_y_new,
                                  cv=5, scoring='neg_mean_squared_error')
    
    # 교차검증 평균 점수 리턴
    return np.mean(scores)


# Optuna study 생성
study = optuna.create_study(direction='minimize')

# study 실행 (n_trials는 시도 횟수)
study.optimize(objective, n_trials=10)

# 최적화된 하이퍼파라미터 값 출력
print(study.best_params)
# Print best hyperparameters and auc

print(f'Best hyperparameters: {study.best_params}')
print(f'Best RMSE: {study.best_value:.4f}')
(est,depth,rate)=study.best_params.values()

def bestreg_parametertuning(rate,depth,est,test_data_columns,y):
    best_lgbmreg=LGBMRegressor(learning_rate=rate,max_depth=depth,n_estimators=est,random_state=42)
    best_lgbmreg.fit(train_X[test_data_columns], train_X[y])
    new_test_data_X[y]=best_lgbmreg.predict(new_test_data_X[test_data_columns])

bestreg_parametertuning(rate,depth,est,test_data_columns,'0.28')

[I 2023-04-26 15:24:40,849] A new study created in memory with name: no-name-8b4e3706-51ae-4789-a612-05ee6d71bc88
[I 2023-04-26 15:24:43,259] Trial 0 finished with value: 0.14406922470325362 and parameters: {'n_estimators': 967, 'max_depth': 6, 'learning_rate': 0.01385032417962858}. Best is trial 0 with value: 0.14406922470325362.
[I 2023-04-26 15:24:43,820] Trial 1 finished with value: 0.14934693385334918 and parameters: {'n_estimators': 687, 'max_depth': 3, 'learning_rate': 0.012005888631085334}. Best is trial 0 with value: 0.14406922470325362.
[I 2023-04-26 15:24:45,331] Trial 2 finished with value: 0.14457410800409634 and parameters: {'n_estimators': 767, 'max_depth': 6, 'learning_rate': 0.020763617020515997}. Best is trial 0 with value: 0.14406922470325362.
[I 2023-04-26 15:24:45,594] Trial 3 finished with value: 0.14705539928005557 and parameters: {'n_estimators': 245, 'max_depth': 4, 'learning_rate': 0.04029889476382111}. Best is trial 0 with value: 0.14406922470325362.
[I 2023-

{'n_estimators': 967, 'max_depth': 6, 'learning_rate': 0.01385032417962858}
Best hyperparameters: {'n_estimators': 967, 'max_depth': 6, 'learning_rate': 0.01385032417962858}
Best RMSE: 0.1441


In [8]:
new_test_data_X

,0.7,278,61,0,0.34,0.30,0.5,0.44,0.40,0.778,0.24,1.93,0.39,1.29,0.43,0.2,0.36,0.17,0.20,0.28
0,0.00,3,1,0.00,0.0,0.0,0.00,0.000,0.0,0.000,0.0,0.00,0.0,0.00,0.0,0.00,0.0,0.00,0.0,0.002779
1,0.00,32,2,0.71,0.0,0.0,0.00,0.000,0.0,0.000,0.0,1.43,0.0,0.00,0.0,0.00,0.0,0.00,0.0,-0.034433
2,0.00,133,51,0.00,0.0,0.0,0.00,0.000,0.0,0.000,0.0,0.00,0.0,0.00,0.0,0.00,0.0,1.09,0.0,0.194212
3,1.26,290,287,0.00,0.0,0.0,1.26,0.000,0.0,0.537,0.0,2.53,0.0,0.00,0.0,0.00,0.0,0.00,0.0,0.038015
4,0.00,23,3,0.00,0.0,0.0,0.00,0.000,0.0,0.000,0.0,3.75,0.0,0.00,0.0,0.00,0.0,0.00,0.0,0.044489
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
915,0.14,898,217,0.14,0.0,0.0,1.47,0.215,0.0,1.004,0.0,2.35,0.0,1.62,0.0,0.29,0.0,0.00,0.0,0.010297
916,0.00,740,54,0.09,0.0,0.0,0.09,0.979,0.0,5.213,0.0,3.75,0.0,0.00,0.0,0.19,0.0,0.00,0.0,-0.081908
917,0.00,3,1,0.00,0.0,0.0,0.00,0.000,0.0,0.000,0.0,0.00,0.0,0.00,0.0,0.00,0.0,0.00,0.0,0.002779
918,0.00,92,11,0.00,0.0,0.0,0.00,0.000,0.0,0.000,0.0,1.12,0.0,0.00,0.0,0.00,0.0,0.56,0.0,0.082900


In [9]:
y_values = ['0.21', '0.32.1', '0.41', '0.29', '3.756', '0.35', '0.25', '0.15', 
                        '0.64.1', '0.38', '0.45', '0.6', '0.33', '0.42', '0.14', '0.9', '0.10', 
                        '0.3', '0.13', '0.8', '0.19', '0.23', '0.18', '0.22', '0.16', '0.64.2', 
                        '0.1', '0.31', '0.27', '0.11', '0.4', '0.12', '0.26', '0.64', '0.37', '0.32', '0.96']  # y 값을 포함하는 리스트

for y in y_values:
    train_X_new = train_X.loc[:, test_data_columns]
    train_y_new = train_X.loc[:, y]
    study = optuna.create_study(direction='minimize')

    # study 실행 (n_trials는 시도 횟수)
    study.optimize(objective, n_trials=10)

    # 최적화된 하이퍼파라미터 값 출력
    print("For y =", y)
    print(study.best_params)

    (est, depth, rate) = study.best_params.values()
    print(rate, depth, est)

    bestreg_parametertuning(rate, depth, est, test_data_columns, y)

[I 2023-04-26 15:26:15,275] A new study created in memory with name: no-name-220f8d79-6f84-4287-ad03-d178c398b61e
[I 2023-04-26 15:26:16,464] Trial 0 finished with value: 0.11310878690909139 and parameters: {'n_estimators': 648, 'max_depth': 8, 'learning_rate': 0.017830814995521777}. Best is trial 0 with value: 0.11310878690909139.
[I 2023-04-26 15:26:18,444] Trial 1 finished with value: 0.1177235940134771 and parameters: {'n_estimators': 955, 'max_depth': 8, 'learning_rate': 0.08187911297976688}. Best is trial 0 with value: 0.11310878690909139.
[I 2023-04-26 15:26:19,583] Trial 2 finished with value: 0.11264144655996913 and parameters: {'n_estimators': 643, 'max_depth': 7, 'learning_rate': 0.026280161387647848}. Best is trial 2 with value: 0.11264144655996913.
[I 2023-04-26 15:26:19,804] Trial 3 finished with value: 0.11336486306456495 and parameters: {'n_estimators': 163, 'max_depth': 5, 'learning_rate': 0.06800679628998191}. Best is trial 2 with value: 0.11264144655996913.
[I 2023-0

For y = 0.21
{'n_estimators': 643, 'max_depth': 7, 'learning_rate': 0.026280161387647848}
0.026280161387647848 7 643


[I 2023-04-26 15:26:24,757] A new study created in memory with name: no-name-9f831a47-118f-470a-b558-f56bb61be2c4
[I 2023-04-26 15:26:25,188] Trial 0 finished with value: 0.6356806978330608 and parameters: {'n_estimators': 577, 'max_depth': 3, 'learning_rate': 0.09194818999947596}. Best is trial 0 with value: 0.6356806978330608.
[I 2023-04-26 15:26:26,102] Trial 1 finished with value: 0.6289395149787586 and parameters: {'n_estimators': 968, 'max_depth': 4, 'learning_rate': 0.020372847463071893}. Best is trial 1 with value: 0.6289395149787586.
[I 2023-04-26 15:26:27,037] Trial 2 finished with value: 0.6432645898910188 and parameters: {'n_estimators': 757, 'max_depth': 5, 'learning_rate': 0.09433430878434203}. Best is trial 1 with value: 0.6289395149787586.
[I 2023-04-26 15:26:27,629] Trial 3 finished with value: 0.6229015895653396 and parameters: {'n_estimators': 235, 'max_depth': 8, 'learning_rate': 0.014645103951150115}. Best is trial 3 with value: 0.6229015895653396.
[I 2023-04-26 15

For y = 0.32.1
{'n_estimators': 295, 'max_depth': 8, 'learning_rate': 0.028049175367755544}
0.028049175367755544 8 295


[I 2023-04-26 15:26:33,608] Trial 0 finished with value: 0.028588595999779653 and parameters: {'n_estimators': 822, 'max_depth': 8, 'learning_rate': 0.02042240535047624}. Best is trial 0 with value: 0.028588595999779653.
[I 2023-04-26 15:26:33,824] Trial 1 finished with value: 0.0380069552690906 and parameters: {'n_estimators': 242, 'max_depth': 3, 'learning_rate': 0.017861479153952694}. Best is trial 0 with value: 0.028588595999779653.
[I 2023-04-26 15:26:34,316] Trial 2 finished with value: 0.03687842572633389 and parameters: {'n_estimators': 630, 'max_depth': 3, 'learning_rate': 0.010469196424766168}. Best is trial 0 with value: 0.028588595999779653.
[I 2023-04-26 15:26:34,677] Trial 3 finished with value: 0.03212871247378287 and parameters: {'n_estimators': 326, 'max_depth': 4, 'learning_rate': 0.035184627700520756}. Best is trial 0 with value: 0.028588595999779653.
[I 2023-04-26 15:26:35,005] Trial 4 finished with value: 0.03845805690507594 and parameters: {'n_estimators': 414, 'm

...

For y = 0.37
{'n_estimators': 102, 'max_depth': 8, 'learning_rate': 0.018723886903277403}
0.018723886903277403 8 102


[I 2023-04-26 15:31:28,670] Trial 0 finished with value: 0.40407772116230367 and parameters: {'n_estimators': 449, 'max_depth': 9, 'learning_rate': 0.012078367892692144}. Best is trial 0 with value: 0.40407772116230367.
[I 2023-04-26 15:31:29,461] Trial 1 finished with value: 0.41343155076357174 and parameters: {'n_estimators': 713, 'max_depth': 4, 'learning_rate': 0.01341121425137411}. Best is trial 0 with value: 0.40407772116230367.
[I 2023-04-26 15:31:30,456] Trial 2 finished with value: 0.417695224393866 and parameters: {'n_estimators': 959, 'max_depth': 3, 'learning_rate': 0.011803748372307854}. Best is trial 0 with value: 0.40407772116230367.
[I 2023-04-26 15:31:30,751] Trial 3 finished with value: 0.41368319933707653 and parameters: {'n_estimators': 316, 'max_depth': 4, 'learning_rate': 0.025507103013668247}. Best is trial 0 with value: 0.40407772116230367.
[I 2023-04-26 15:31:31,619] Trial 4 finished with value: 0.41240582431525474 and parameters: {'n_estimators': 980, 'max_dep

For y = 0.32
{'n_estimators': 519, 'max_depth': 9, 'learning_rate': 0.05941957503017391}
0.05941957503017391 9 519


[I 2023-04-26 15:31:35,988] A new study created in memory with name: no-name-98a1a40c-9bfa-4838-bbb4-2bdfeeb3d3b0
[I 2023-04-26 15:31:36,462] Trial 0 finished with value: 0.8536929792577734 and parameters: {'n_estimators': 379, 'max_depth': 5, 'learning_rate': 0.03870833549605539}. Best is trial 0 with value: 0.8536929792577734.
[I 2023-04-26 15:31:37,992] Trial 1 finished with value: 0.854321252941195 and parameters: {'n_estimators': 682, 'max_depth': 8, 'learning_rate': 0.02777077302741777}. Best is trial 0 with value: 0.8536929792577734.
[I 2023-04-26 15:31:39,341] Trial 2 finished with value: 0.8543097904274598 and parameters: {'n_estimators': 978, 'max_depth': 5, 'learning_rate': 0.023899279428120992}. Best is trial 0 with value: 0.8536929792577734.
[I 2023-04-26 15:31:39,657] Trial 3 finished with value: 0.8668074197403154 and parameters: {'n_estimators': 359, 'max_depth': 3, 'learning_rate': 0.07123900745742082}. Best is trial 0 with value: 0.8536929792577734.
[I 2023-04-26 15:3

For y = 0.96
{'n_estimators': 871, 'max_depth': 6, 'learning_rate': 0.04155294008298901}
0.04155294008298901 6 871


In [10]:
new_test_data_X

,0.7,278,61,0,0.34,0.30,0.5,0.44,0.40,0.778,...,0.31,0.27,0.11,0.4,0.12,0.26,0.64,0.37,0.32,0.96
0,0.00,3,1,0.00,0.0,0.0,0.00,0.000,0.0,0.000,...,0.000933,0.005337,0.000799,0.019632,-0.001841,0.008735,0.005209,0.177928,0.550175,0.018455
1,0.00,32,2,0.71,0.0,0.0,0.00,0.000,0.0,0.000,...,-0.007723,0.016784,0.045578,0.005986,0.149726,-0.008465,0.000696,0.431179,0.140337,0.568936
2,0.00,133,51,0.00,0.0,0.0,0.00,0.000,0.0,0.000,...,0.003049,0.158220,0.109590,0.040135,0.053696,0.283900,-0.007289,0.165669,0.108198,0.136107
3,1.26,290,287,0.00,0.0,0.0,1.26,0.000,0.0,0.537,...,-0.032151,-0.017787,2.941139,0.899773,1.238776,0.009469,-0.038205,0.135875,0.904080,3.704916
4,0.00,23,3,0.00,0.0,0.0,0.00,0.000,0.0,0.000,...,-0.013580,0.006331,0.028926,0.022617,-0.014180,-0.009212,-0.066776,0.535388,0.127661,0.745868
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
915,0.14,898,217,0.14,0.0,0.0,1.47,0.215,0.0,1.004,...,0.224858,-0.033515,0.099559,0.638000,0.411254,0.020380,0.538370,0.104851,0.298272,2.026425
916,0.00,740,54,0.09,0.0,0.0,0.09,0.979,0.0,5.213,...,0.014636,-0.012334,0.117354,0.032608,0.084607,0.015811,0.540825,0.425974,0.385703,1.282773
917,0.00,3,1,0.00,0.0,0.0,0.00,0.000,0.0,0.000,...,0.000933,0.005337,0.000799,0.019632,-0.001841,0.008735,0.005209,0.177928,0.550175,0.018455
918,0.00,92,11,0.00,0.0,0.0,0.00,0.000,0.0,0.000,...,0.012103,0.221773,0.006330,0.020667,0.047490,0.139738,0.055540,0.169529,0.281917,0.379545


In [11]:
newtestdata=pd.concat([new_test_data_X,new_test_data_y],axis=1)

In [12]:
newtestdata.head(5)

,0.7,278,61,0,0.34,0.30,0.5,0.44,0.40,0.778,...,0.27,0.11,0.4,0.12,0.26,0.64,0.37,0.32,0.96,1
0,0.00,3,1,0.00,0.0,0.0,0.00,0.0,0.0,0.000,...,0.005337,0.000799,0.019632,-0.001841,0.008735,0.005209,0.177928,0.550175,0.018455,0
1,0.00,32,2,0.71,0.0,0.0,0.00,0.0,0.0,0.000,...,0.016784,0.045578,0.005986,0.149726,-0.008465,0.000696,0.431179,0.140337,0.568936,0
2,0.00,133,51,0.00,0.0,0.0,0.00,0.0,0.0,0.000,...,0.158220,0.109590,0.040135,0.053696,0.283900,-0.007289,0.165669,0.108198,0.136107,0
3,1.26,290,287,0.00,0.0,0.0,1.26,0.0,0.0,0.537,...,-0.017787,2.941139,0.899773,1.238776,0.009469,-0.038205,0.135875,0.904080,3.704916,1
4,0.00,23,3,0.00,0.0,0.0,0.00,0.0,0.0,0.000,...,0.006331,0.028926,0.022617,-0.014180,-0.009212,-0.066776,0.535388,0.127661,0.745868,0


In [13]:
import lightgbm as lgb

In [14]:
def objective(trial):
    # Define hyperparameters to optimize 
    params={
        'boosting_type':'gbdt',
        'objective':'binary',
        'metric':'binary_logloss',
        'learning_rate':trial.suggest_loguniform('learning_rate',0.01,0.1),
        'max_depth':trial.suggest_int('max_depth',3,9),
        'n_estimators':trial.suggest_int("n_estimators",100,1000)
    }
    # Train and evaluate model 
    lgb_cv=lgb.LGBMClassifier(**params, random_state=42,scale_pos_weight=ratio)
    scores=cross_val_score(lgb_cv,train_X,train_y,cv=5,scoring='roc_auc')
    auc=scores.mean()
    return auc 

# Define study object and optimize 

study=optuna.create_study(direction='maximize',study_name='lgb_boost_opt',load_if_exists=True)
study.optimize(objective, n_trials=10)

# Print best hyperparameters and auc
print(f'Best hyperparameters: {study.best_params}')
print(f'Best AUC: {study.best_value:.4f}')

[I 2023-04-26 15:36:10,979] A new study created in memory with name: lgb_boost_opt
[I 2023-04-26 15:36:11,170] Trial 0 finished with value: 0.9845141872453762 and parameters: {'learning_rate': 0.05149450129162212, 'max_depth': 3, 'n_estimators': 111}. Best is trial 0 with value: 0.9845141872453762.
[I 2023-04-26 15:36:13,381] Trial 1 finished with value: 0.99031537304876 and parameters: {'learning_rate': 0.028177994367239483, 'max_depth': 5, 'n_estimators': 947}. Best is trial 1 with value: 0.99031537304876.
[I 2023-04-26 15:36:15,268] Trial 2 finished with value: 0.9889534041921169 and parameters: {'learning_rate': 0.013004985445763399, 'max_depth': 5, 'n_estimators': 725}. Best is trial 1 with value: 0.99031537304876.
[I 2023-04-26 15:36:15,481] Trial 3 finished with value: 0.9724628421792714 and parameters: {'learning_rate': 0.012015590998075388, 'max_depth': 3, 'n_estimators': 134}. Best is trial 1 with value: 0.99031537304876.
[I 2023-04-26 15:36:16,209] Trial 4 finished with valu

Best hyperparameters: {'learning_rate': 0.028177994367239483, 'max_depth': 5, 'n_estimators': 947}
Best AUC: 0.9903


In [15]:
lgb=lgb.LGBMClassifier(learning_rate= 0.028177994367239483,max_depth= 5,n_estimators=947,scale_pos_weight=ratio,random_state=42)

In [16]:
lgb.fit(train_X,train_y)

LGBMClassifier(learning_rate=0.028177994367239483, max_depth=5,
               n_estimators=947, random_state=42,
               scale_pos_weight=1.5879043600562588)

In [17]:
pred=lgb.predict_proba(new_test_data_X)[:,1]

In [18]:
from sklearn.metrics import accuracy_score, precision_score, recall_score, confusion_matrix, f1_score

def get_clf_prob(y_test, probability):
  pred=np.where(probability > 0.50,1,0)
  confusion=confusion_matrix(y_test, pred)
  accuracy=accuracy_score(y_test,pred)
  precision=precision_score(y_test,pred) 
  recall=recall_score(y_test,pred) 
  # F1 스코어 추가 
  f1=f1_score(y_test,pred,average='macro')
  Roc_score=roc_auc_score(y_test,probability)
  print('임계값: ', 0.5) 
  print('오차행렬')
  print(confusion) 
  # f1 score print 추가 
  print('정확도: {0:.4f}, 정밀도: {1:.4f}, 재현율: {2:.4f}, F1:{3:.4f}, AUC:{4: .4f}'.format(accuracy,precision,recall,f1,Roc_score))

In [19]:
get_clf_prob(new_test_data_y,pred) 

임계값:  0.5
오차행렬
[[521   9]
 [163 227]]
정확도: 0.8130, 정밀도: 0.9619, 재현율: 0.5821, F1:0.7918, AUC: 0.9297
